# GUESSING THE GROUND WATER LEVEL 

calling the libraries

In [54]:
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing and feature selection
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

# For handling imbalanced datasets (if needed)
#from imblearn.over_sampling import SMOTE

# Modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix



## PRE PROCESSING and FEATURES ENGINEERING

importing Train data

In [55]:
path=('X_train_Hi5.csv')

df1=pd.read_csv(path,nrows=10000,low_memory=False)
threshold = 0.6

# Calculate the percentage of missing values in each column
missing_percent = df1.isnull().median()


# Drop columns with missing values > threshold
columns_to_drop = missing_percent[missing_percent > threshold].index
df = df1.drop(columns=columns_to_drop)

replacing missing values

In [56]:
# Separate numerical and categorical columns
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

# Impute numerical columns
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])
df

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
0,0,01,Sun Jul 14 13:00:02 CEST 2024,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,...,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,25250,2.9,16.2,High
1,1,01,Sun Jul 14 13:00:02 CEST 2024,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,...,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,24660,44.5,11,Very High
2,2,01,Sun Jul 14 13:00:02 CEST 2024,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,...,5.085060,45.812828,69266.0,3.810492e+08,0,8085.0,24890,8.4,7.8,High
3,3,01,Sun Jul 14 13:00:02 CEST 2024,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,...,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,39700,2.4,5.2,Very High
4,4,01,Sun Jul 14 13:00:02 CEST 2024,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,...,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,26180,21.5,9.8,Very Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,10517,06,Fri Jun 28 07:31:38 CEST 2024,37.70,Alpes-Maritimes,06027,collège Romée,['121BB01'],10.00,09994X0536/PZ_CRO,...,7.193913,43.854155,6021.0,1.611474e+08,0.1,52580.0,23940,5.3,7.8,Low
9996,10518,06,Fri Jun 28 07:31:38 CEST 2024,37.70,Alpes-Maritimes,06161,Grand Pré,['121BB01'],14.00,09994X0533/PZ_GDP,...,7.193913,43.854155,6021.0,1.621418e+08,0.1,16779.0,26260,5.2,6,Low
9997,10519,06,Fri Jun 28 07:31:38 CEST 2024,37.70,Alpes-Maritimes,06027,Hippodrome,['121BB01'],1.00,09998X0197/PZ_HIP,...,7.193913,43.854155,6021.0,1.566103e+08,0.1,52580.0,23940,5.3,7.8,High
9998,10520,06,Fri Jun 28 07:31:38 CEST 2024,17.50,Alpes-Maritimes,06033,P33BIS,['121BB01'],76.17,09735X0161/P33BIS,...,7.188727,43.901261,6078.0,1.036715e+09,0,13277.0,23480,35.8,9.5,Very High


manually encoding the Target  piezo_groundwater_level_category and dropping it from train data

In [57]:
df['piezo_groundwater_level_category'] = df['piezo_groundwater_level_category']

# Step 2: Define the mapping
category_mapping = {
    'Very High': 4,
    'High': 3,
    'Average': 2,
    'Low': 1,
    'Very Low': 0
}

# Step 3: Apply the mapping
df['piezo_groundwater_level_category'] = df['piezo_groundwater_level_category'].map(category_mapping)
y=df['piezo_groundwater_level_category']
df=df.drop(['piezo_groundwater_level_category'], axis=1)
df

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_volume_obtention_mode_label_2,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const
0,0,01,Sun Jul 14 13:00:02 CEST 2024,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,...,Mesure indirecte,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,25250,2.9,16.2
1,1,01,Sun Jul 14 13:00:02 CEST 2024,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,...,Mesure indirecte,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,24660,44.5,11
2,2,01,Sun Jul 14 13:00:02 CEST 2024,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,...,Mesure indirecte,5.085060,45.812828,69266.0,3.810492e+08,0,8085.0,24890,8.4,7.8
3,3,01,Sun Jul 14 13:00:02 CEST 2024,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,...,Mesure indirecte,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,39700,2.4,5.2
4,4,01,Sun Jul 14 13:00:02 CEST 2024,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,...,Mesure indirecte,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,26180,21.5,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,10517,06,Fri Jun 28 07:31:38 CEST 2024,37.70,Alpes-Maritimes,06027,collège Romée,['121BB01'],10.00,09994X0536/PZ_CRO,...,Mesure indirecte,7.193913,43.854155,6021.0,1.611474e+08,0.1,52580.0,23940,5.3,7.8
9996,10518,06,Fri Jun 28 07:31:38 CEST 2024,37.70,Alpes-Maritimes,06161,Grand Pré,['121BB01'],14.00,09994X0533/PZ_GDP,...,Mesure indirecte,7.193913,43.854155,6021.0,1.621418e+08,0.1,16779.0,26260,5.2,6
9997,10519,06,Fri Jun 28 07:31:38 CEST 2024,37.70,Alpes-Maritimes,06027,Hippodrome,['121BB01'],1.00,09998X0197/PZ_HIP,...,Mesure indirecte,7.193913,43.854155,6021.0,1.566103e+08,0.1,52580.0,23940,5.3,7.8
9998,10520,06,Fri Jun 28 07:31:38 CEST 2024,17.50,Alpes-Maritimes,06033,P33BIS,['121BB01'],76.17,09735X0161/P33BIS,...,Mesure indirecte,7.188727,43.901261,6078.0,1.036715e+09,0,13277.0,23480,35.8,9.5


Importing Test Data

In [58]:
path=('X_test_Hi5.csv')


df_test = pd.read_csv(path, low_memory=False)
df_test# Drop columns with missing values > threshold

df_test= df_test.drop(columns=columns_to_drop)


In [59]:
df_test

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_volume_obtention_mode_label_2,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const
0,2331795,01,Sun Jul 14 13:00:02 CEST 2024,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,...,NaN,NaN,NaN,NaN,0.0,11.8,992.0,25250,2.9,16.2
1,2331796,01,Sun Jul 14 13:00:02 CEST 2024,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,...,NaN,NaN,NaN,NaN,0.0,0.6,1786.0,24660,44.5,11
2,2331797,01,Sun Jul 14 13:00:02 CEST 2024,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,...,NaN,NaN,NaN,NaN,0.0,0,8085.0,24890,8.4,7.8
3,2331798,01,Sun Jul 14 13:00:02 CEST 2024,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,...,NaN,NaN,NaN,NaN,0.0,1.5,2838.0,39700,2.4,5.2
4,2331799,01,Sun Jul 14 13:00:02 CEST 2024,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,...,NaN,NaN,NaN,NaN,0.0,0.2,1352.0,26180,21.5,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611203,3610818,95,Thu Sep 19 11:45:47 CEST 2024,24.00,Val-d'Oise,95119,Puits du hameau de Buchet (Buhy - 95),['121AZ01'],63.00,01258X0020/S1,...,NaN,NaN,NaN,NaN,NaN,27.3,322.0,25850,0,27.3
611204,3610819,95,Thu Sep 19 14:41:55 CEST 2024,25.50,Val-d'Oise,95142,PIÉZOMÈTRE DU HAMEAU DE BERCAGNY (CHARS -95),['113AQ27'],103.00,01266X1013/S1,...,NaN,NaN,NaN,NaN,NaN,1.1,2028.0,25320,13.8,2.9
611205,3610820,95,Thu Sep 19 14:29:16 CEST 2024,30.00,Val-d'Oise,95610,Piézomètre de la place Roland Vasseur (Théméri...,['121AP01'],83.00,01522X0069/PZ2008,...,NaN,NaN,NaN,NaN,NaN,1.6,295.0,31340,4.8,4.8
611206,3610821,95,Thu Dec 14 19:02:16 CET 2023,630.00,Val-d'Oise,95313,PIÉZOMÈTRE DE L'ISLE-ADAM (ISLE-ADAM - 95),['127AA99'],24.00,01531X0047/F,...,NaN,NaN,NaN,NaN,NaN,0.2,11846.0,28610,4.9,6.7


Concatenate train and test data

In [60]:
df_all = pd.concat([df, df_test], axis=0, ignore_index=True)
df_all=df_all.drop(['row_index','piezo_station_update_date'],axis=1)

In [61]:
df_all

,piezo_station_department_code,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,piezo_station_commune_name,piezo_station_longitude,...,prelev_volume_obtention_mode_label_2,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const
0,01,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,Ceyzérieu,5.748241,...,Mesure indirecte,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,25250,2.9,16.2
1,01,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,Saint-Jean-le-Vieux,5.356637,...,Mesure indirecte,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,24660,44.5,11
2,01,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,Meximieux,5.220795,...,Mesure indirecte,5.085060,45.812828,69266.0,3.810492e+08,0,8085.0,24890,8.4,7.8
3,01,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,Péron,5.948977,...,Mesure indirecte,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,39700,2.4,5.2
4,01,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,Tossiat,5.313353,...,Mesure indirecte,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,26180,21.5,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621203,95,24.00,Val-d'Oise,95119,Puits du hameau de Buchet (Buhy - 95),['121AZ01'],63.00,01258X0020/S1,Buhy,1.711584,...,NaN,NaN,NaN,NaN,NaN,27.3,322.0,25850,0,27.3
621204,95,25.50,Val-d'Oise,95142,PIÉZOMÈTRE DU HAMEAU DE BERCAGNY (CHARS -95),['113AQ27'],103.00,01266X1013/S1,Chars,1.901276,...,NaN,NaN,NaN,NaN,NaN,1.1,2028.0,25320,13.8,2.9
621205,95,30.00,Val-d'Oise,95610,Piézomètre de la place Roland Vasseur (Théméri...,['121AP01'],83.00,01522X0069/PZ2008,Théméricourt,1.897576,...,NaN,NaN,NaN,NaN,NaN,1.6,295.0,31340,4.8,4.8
621206,95,630.00,Val-d'Oise,95313,PIÉZOMÈTRE DE L'ISLE-ADAM (ISLE-ADAM - 95),['127AA99'],24.00,01531X0047/F,L'Isle-Adam,2.209806,...,NaN,NaN,NaN,NaN,NaN,0.2,11846.0,28610,4.9,6.7


Transforming the insee percentages to float64 from object

In [62]:
import pandas as pd

# Define the list of columns to convert
columns_to_convert = ['insee_%_agri', 'insee_%_ind', 'insee_%_const']

# Create a copy of the DataFrame
df_selected1 = df_all.copy()

# Inspect the initial data types and values
print("Initial data types:")
print(df_selected1[columns_to_convert].dtypes)

print("\nSample of data before conversion:")
print(df_selected1[columns_to_convert].head())

# Convert columns to numeric and inspect problematic data
for col in columns_to_convert:
    if col in df_selected1.columns:
        df_selected1[col] = pd.to_numeric(df_selected1[col], errors='coerce')
        # Identify rows where conversion failed
        invalid_rows = df_selected1[df_selected1[col].isna() & df1[col].notna()]
        if not invalid_rows.empty:
            print(f"\nInvalid data in column '{col}':")
            print(invalid_rows[[col]])

# Display data types after conversion
print("\nData types after conversion:")
print(df_selected1[columns_to_convert].dtypes)

Initial data types:
insee_%_agri     object
insee_%_ind      object
insee_%_const    object
dtype: object

Sample of data before conversion:
  insee_%_agri insee_%_ind insee_%_const
0         11.8         2.9          16.2
1          0.6        44.5            11
2            0         8.4           7.8
3          1.5         2.4           5.2
4          0.2        21.5           9.8

Invalid data in column 'insee_%_agri':
      insee_%_agri
140            NaN
1847           NaN
2609           NaN
4311           NaN
5071           NaN
6776           NaN
7540           NaN
9249           NaN

Invalid data in column 'insee_%_ind':
      insee_%_ind
140           NaN
1847          NaN
2609          NaN
4311          NaN
5071          NaN
6776          NaN
7540          NaN
9249          NaN

Invalid data in column 'insee_%_const':
      insee_%_const
140             NaN
1847            NaN
2609            NaN
4311            NaN
5071            NaN
6776            NaN
7540            NaN


In [63]:
df_selected1

,piezo_station_department_code,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,piezo_station_commune_name,piezo_station_longitude,...,prelev_volume_obtention_mode_label_2,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const
0,01,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,Ceyzérieu,5.748241,...,Mesure indirecte,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,25250,2.9,16.2
1,01,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,Saint-Jean-le-Vieux,5.356637,...,Mesure indirecte,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,24660,44.5,11.0
2,01,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,Meximieux,5.220795,...,Mesure indirecte,5.085060,45.812828,69266.0,3.810492e+08,0.0,8085.0,24890,8.4,7.8
3,01,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,Péron,5.948977,...,Mesure indirecte,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,39700,2.4,5.2
4,01,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,Tossiat,5.313353,...,Mesure indirecte,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,26180,21.5,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621203,95,24.00,Val-d'Oise,95119,Puits du hameau de Buchet (Buhy - 95),['121AZ01'],63.00,01258X0020/S1,Buhy,1.711584,...,NaN,NaN,NaN,NaN,NaN,27.3,322.0,25850,0.0,27.3
621204,95,25.50,Val-d'Oise,95142,PIÉZOMÈTRE DU HAMEAU DE BERCAGNY (CHARS -95),['113AQ27'],103.00,01266X1013/S1,Chars,1.901276,...,NaN,NaN,NaN,NaN,NaN,1.1,2028.0,25320,13.8,2.9
621205,95,30.00,Val-d'Oise,95610,Piézomètre de la place Roland Vasseur (Théméri...,['121AP01'],83.00,01522X0069/PZ2008,Théméricourt,1.897576,...,NaN,NaN,NaN,NaN,NaN,1.6,295.0,31340,4.8,4.8
621206,95,630.00,Val-d'Oise,95313,PIÉZOMÈTRE DE L'ISLE-ADAM (ISLE-ADAM - 95),['127AA99'],24.00,01531X0047/F,L'Isle-Adam,2.209806,...,NaN,NaN,NaN,NaN,NaN,0.2,11846.0,28610,4.9,6.7


formatting the (piezo_measurement_date) feature to show only the month

In [64]:
import re

# Define your date columns
date_cols = ['piezo_measurement_date']#, 'meteo_date', 'hydro_observation_date_elab']

# Function to remove unrecognized timezone abbreviations
def remove_timezone_abbreviation(date_str):
    if pd.isnull(date_str):
        return date_str
    # Remove timezone abbreviations (e.g., 'CEST', 'CET', 'PST', etc.)
    date_str = re.sub(r'\b[A-Z]{3,4}\b', '', date_str)
    return date_str.strip()

# Clean and parse the date columns
for col in date_cols:
    # Remove timezone abbreviations
    df_selected1[col] = df_selected1[col].astype(str).apply(remove_timezone_abbreviation)
    # Parse the cleaned date strings into datetime objects
    df_selected1[col] = pd.to_datetime(df_selected1[col], errors='coerce')



df_selected1['piezo_measurement_date']=df_selected1['piezo_measurement_date'].dt.month

df_selected1

,piezo_station_department_code,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,piezo_station_commune_name,piezo_station_longitude,...,prelev_volume_obtention_mode_label_2,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const
0,01,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,Ceyzérieu,5.748241,...,Mesure indirecte,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,25250,2.9,16.2
1,01,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,Saint-Jean-le-Vieux,5.356637,...,Mesure indirecte,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,24660,44.5,11.0
2,01,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,Meximieux,5.220795,...,Mesure indirecte,5.085060,45.812828,69266.0,3.810492e+08,0.0,8085.0,24890,8.4,7.8
3,01,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,Péron,5.948977,...,Mesure indirecte,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,39700,2.4,5.2
4,01,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,Tossiat,5.313353,...,Mesure indirecte,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,26180,21.5,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621203,95,24.00,Val-d'Oise,95119,Puits du hameau de Buchet (Buhy - 95),['121AZ01'],63.00,01258X0020/S1,Buhy,1.711584,...,NaN,NaN,NaN,NaN,NaN,27.3,322.0,25850,0.0,27.3
621204,95,25.50,Val-d'Oise,95142,PIÉZOMÈTRE DU HAMEAU DE BERCAGNY (CHARS -95),['113AQ27'],103.00,01266X1013/S1,Chars,1.901276,...,NaN,NaN,NaN,NaN,NaN,1.1,2028.0,25320,13.8,2.9
621205,95,30.00,Val-d'Oise,95610,Piézomètre de la place Roland Vasseur (Théméri...,['121AP01'],83.00,01522X0069/PZ2008,Théméricourt,1.897576,...,NaN,NaN,NaN,NaN,NaN,1.6,295.0,31340,4.8,4.8
621206,95,630.00,Val-d'Oise,95313,PIÉZOMÈTRE DE L'ISLE-ADAM (ISLE-ADAM - 95),['127AA99'],24.00,01531X0047/F,L'Isle-Adam,2.209806,...,NaN,NaN,NaN,NaN,NaN,0.2,11846.0,28610,4.9,6.7


Label encoding 

In [65]:
label_encoders = {}
for col in df_selected1.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_selected1[col] = le.fit_transform(df_selected1[col])
    label_encoders[col] = le  # Save encoders for inverse_transform if needed
df_selected1.dtypes

piezo_station_department_code          int32
piezo_station_investigation_depth    float64
piezo_station_department_name          int32
piezo_station_commune_code_insee       int32
piezo_station_pe_label                 int32
                                      ...   
insee_%_agri                         float64
insee_pop_commune                    float64
insee_med_living_level                 int32
insee_%_ind                          float64
insee_%_const                        float64
Length: 89, dtype: object

splitting the train and test data back after encoding

In [66]:
df_original = df_selected1.iloc[:10000]  # The first 2,830,316 rows
df_test_original = df_selected1.iloc[10000:]  # The remaining rows

check for correlation

In [67]:
corr_matrix = df_original.corr()
print(corr_matrix)

                                   piezo_station_department_code  \
piezo_station_department_code                           1.000000   
piezo_station_investigation_depth                      -0.016232   
piezo_station_department_name                           0.848570   
piezo_station_commune_code_insee                        0.995822   
piezo_station_pe_label                                  0.048603   
...                                                          ...   
insee_%_agri                                           -0.048951   
insee_pop_commune                                      -0.009694   
insee_med_living_level                                  0.105053   
insee_%_ind                                            -0.010724   
insee_%_const                                           0.011398   

                                   piezo_station_investigation_depth  \
piezo_station_department_code                              -0.016232   
piezo_station_investigation_depth      

In [68]:
# Create a mask to ignore self-correlations
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Apply the mask and flatten the correlation matrix
corr_pairs = corr_matrix.where(mask).stack().reset_index()
corr_pairs.columns = ['Feature1', 'Feature2', 'Correlation']

# Filter pairs with correlation greater than 0.9
high_corr = corr_pairs[corr_pairs['Correlation'] > 0.8]
print(high_corr)

                           Feature1                          Feature2  \
0     piezo_station_department_code     piezo_station_department_code   
2     piezo_station_department_code     piezo_station_department_name   
3     piezo_station_department_code  piezo_station_commune_code_insee   
65    piezo_station_department_code       prelev_commune_code_insee_0   
72    piezo_station_department_code       prelev_commune_code_insee_1   
...                             ...                               ...   
3726                   insee_%_agri                      insee_%_agri   
3731              insee_pop_commune                 insee_pop_commune   
3735         insee_med_living_level            insee_med_living_level   
3738                    insee_%_ind                       insee_%_ind   
3740                  insee_%_const                     insee_%_const   

      Correlation  
0        1.000000  
2        0.848570  
3        0.995822  
65       0.804620  
72       0.812654  
...

removing highly correlated features

In [69]:
# Create a set to hold the columns to drop
df_original['piezo_groundwater_level_category']=y
up_sampling_df=df_original[(df_original['piezo_measurement_date'] >= 5) & (df_original['piezo_measurement_date'] <= 8)]

to_drop = set()

for index, row in high_corr.iterrows():
    feature1 = row['Feature1']
    feature2 = row['Feature2']
        # Add one of the features to the to_drop set
    if feature1 != feature2 :
        if feature1 == 'piezo_mesurement_date':
            to_drop.add(feature2)
        else:
            to_drop.add(feature1)

up_sampling_df

C:\Users\PC\AppData\Local\Temp\ipykernel_6312\4019583181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_original['piezo_groundwater_level_category']=y


,piezo_station_department_code,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,piezo_station_commune_name,piezo_station_longitude,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category


dropping the highly coorelated variables from the data

In [70]:
# Drop the columns
df_original.drop(columns=list(to_drop), inplace=True)
df_test_original.drop(columns=list(to_drop), inplace=True)
up_sampling_df.drop(columns=list(to_drop), inplace=True)
df_original


C:\Users\PC\AppData\Local\Temp\ipykernel_6312\3251193586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_original.drop(columns=list(to_drop), inplace=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_6312\3251193586.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_original.drop(columns=list(to_drop), inplace=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_6312\3251193586.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

,piezo_station_investigation_depth,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_commune_name,piezo_bss_code,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_name,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
0,20.00,1526,741,394,1560,1,1,2,0,1,...,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,628,2.9,16.2,3
1,35.60,1518,756,1730,1465,1,1,2,0,1,...,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,579,44.5,11.0,4
2,35.22,1506,15,1220,1556,1,1,2,0,1,...,5.085060,45.812828,69266.0,3.810492e+08,0.0,8085.0,597,8.4,7.8,3
3,34.20,1519,581,1556,1416,1,1,2,0,1,...,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,920,2.4,5.2,4
4,37.30,601,565,2034,1415,1,1,2,0,1,...,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,703,21.5,9.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,37.70,2424,156,337,2470,1,1,1,0,1,...,7.193913,43.854155,6021.0,1.611474e+08,0.1,52580.0,515,5.3,7.8,1
9996,37.70,931,156,2165,2468,1,1,1,0,1,...,7.193913,43.854155,6021.0,1.621418e+08,0.1,16779.0,709,5.2,6.0,1
9997,37.70,966,156,337,2475,1,1,2,0,1,...,7.193913,43.854155,6021.0,1.566103e+08,0.1,52580.0,515,5.3,7.8,3
9998,17.50,1451,156,359,2335,1,1,1,0,1,...,7.188727,43.901261,6078.0,1.036715e+09,0.0,13277.0,475,35.8,9.5,4


In [71]:
X=df_original

## TRAINING THE MODEL

In [75]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

# Split features and labels
X = df_original.drop(columns=['piezo_groundwater_level_category'])
y = df_original['piezo_groundwater_level_category']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

# Initialize the XGBoost model
model = XGBClassifier(
    objective='multi:softprob',  
    tree_method='hist',
    device="cuda",
    eval_metric='mlogloss',
    n_estimators=4000,
    max_depth=14,
    learning_rate=0.09,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=1,
    reg_alpha=0.1,
    reg_lambda=1,
)

# Cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for train_idx, val_idx in kf.split(X_train, y_train):
    X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    model.fit(
        X_fold_train, y_fold_train,
        eval_set=[(X_fold_val, y_fold_val)],
        verbose=True,
        
    )
    
    y_fold_pred = model.predict(X_fold_val)
    cv_scores.append(accuracy_score(y_fold_val, y_fold_pred))

print(f"Cross-validation Accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

# Final evaluation on the test set
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy: {test_accuracy:.4f}")


[0]	validation_0-mlogloss:1.50899
[1]	validation_0-mlogloss:1.41836
[2]	validation_0-mlogloss:1.34157
[3]	validation_0-mlogloss:1.27074
[4]	validation_0-mlogloss:1.20703
[5]	validation_0-mlogloss:1.15236
[6]	validation_0-mlogloss:1.10186
[7]	validation_0-mlogloss:1.05537
[8]	validation_0-mlogloss:1.01170
[9]	validation_0-mlogloss:0.97185
[10]	validation_0-mlogloss:0.93488
[11]	validation_0-mlogloss:0.89923
[12]	validation_0-mlogloss:0.86825
[13]	validation_0-mlogloss:0.83843
[14]	validation_0-mlogloss:0.81433
[15]	validation_0-mlogloss:0.78978
[16]	validation_0-mlogloss:0.76740
[17]	validation_0-mlogloss:0.74512
[18]	validation_0-mlogloss:0.72329
[19]	validation_0-mlogloss:0.70587
[20]	validation_0-mlogloss:0.69024
[21]	validation_0-mlogloss:0.67499
[22]	validation_0-mlogloss:0.65986
[23]	validation_0-mlogloss:0.64526
[24]	validation_0-mlogloss:0.63284
[25]	validation_0-mlogloss:0.62231
[26]	validation_0-mlogloss:0.61160
[27]	validation_0-mlogloss:0.60165
[28]	validation_0-mlogloss:0.5

KeyboardInterrupt: 

evaluating the model Train data and tesing it's accuracy

In [ ]:
from sklearn.metrics import accuracy_score, classification_report ,f1_score


accuracy_train = accuracy_score(y_test, y_pred)
print(f"Accuracy on training set: {accuracy_train:.2f}")
# Classification report for detailed metrics (precision, recall, F1-score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
print(f"F1 Score (Weighted): {f1:.4f}")

Accuracy on training set: 0.92

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       102
           1       0.94      0.88      0.91       148
           2       0.91      0.90      0.91       218
           3       0.89      0.92      0.91       276
           4       0.94      0.96      0.95       256

    accuracy                           0.92      1000
   macro avg       0.92      0.92      0.92      1000
weighted avg       0.92      0.92      0.92      1000

F1 Score (Weighted): 0.9189


In [ ]:
df_test_original=df_test_original[X_train.columns]

predicting in the Target in the X_test_HI5 csv 

In [ ]:
y_pred_test = model.predict(df_test_original)

reverse encoding the values of the target

In [ ]:
import pandas as pd

# Check unique values in y_pred_test
print("Unique values in y_pred_test:", pd.unique(y_pred_test))

# Define the correct mapping
y_mapping = {
    0: 'Very Low',
    1: 'Low',
    2: 'Average',
    3: 'High',
    4: 'Very High'
}

# Convert y_pred_test to a pandas Series
y_pred_test_series = pd.Series(y_pred_test)

# Apply the mapping
y_encoded_test = y_pred_test_series.map(y_mapping)

# Check for unmapped values
unmapped = y_encoded_test.isnull().sum()
print("Number of unmapped values:", unmapped)

# Print the value of y at a specific index, ensuring proper indexing
index_to_check = 2830315
if index_to_check < len(y):
    print(f"Value at index {index_to_check} in y:", y[index_to_check])
else:
    print(f"Index {index_to_check} out of bounds for y.")

y_encoded_test.name='piezo_groundwater_level_category'
y_encoded_test

Unique values in y_pred_test: [3 4 0 2 1]
Number of unmapped values: 0
Index 2830315 out of bounds for y.


0              High
1         Very High
2              High
3              High
4              High
            ...    
611203     Very Low
611204      Average
611205      Average
611206     Very Low
611207     Very Low
Name: piezo_groundwater_level_category, Length: 611208, dtype: object

Saving the csv file in the correct format

In [ ]:
final_result = pd.concat([df_test['row_index'], y_encoded_test], axis=1)
final_result.to_csv('submit_5_grp_55.csv', index=False)

In [ ]:
feature_importances = model.feature_importances_

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print(importance_df)

                                 Feature  Importance
29                    hydro_station_code    0.045169
50                     prelev_latitude_2    0.030102
11                   piezo_producer_name    0.027775
49                    prelev_longitude_2    0.026764
13                              meteo_id    0.025997
10                   piezo_producer_code    0.024612
4                         piezo_bss_code    0.023668
45               prelev_structure_code_2    0.023430
51           prelev_commune_code_insee_2    0.022309
47                  prelev_usage_label_2    0.021728
43                       prelev_volume_1    0.021601
48  prelev_volume_obtention_mode_label_2    0.021239
2             piezo_station_bdlisa_codes    0.021033
54                     insee_pop_commune    0.020353
52               prelev_other_volume_sum    0.020239
57                         insee_%_const    0.019882
55                insee_med_living_level    0.019753
42               prelev_structure_code_1    0.